In [36]:
! wget https://raw.githubusercontent.com/quickfix/quickfix/master/spec/FIX44.xml

--2021-12-22 02:03:06--  https://raw.githubusercontent.com/quickfix/quickfix/master/spec/FIX44.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 320230 (313K) [text/plain]
Saving to: ‘FIX44.xml.1’

FIX44.xml.1         100%[===================>] 312.72K   800KB/s    in 0.4s    

2021-12-22 02:03:08 (800 KB/s) - ‘FIX44.xml.1’ saved [320230/320230]



In [1]:
import importlib
import quickfix
import myfix
import quickfix44
import fixtrading
import xml.etree.ElementTree as ET
import tqdm

tag_dict = myfix.tag_dict()


class Application(quickfix.Application): 
    
    def __init__(self, verbose = True, xml_name = myfix.DEFAULT_XML):
        super().__init__()
        self.data_dictionary  = quickfix.DataDictionary(xml_name)
        self.field_dict = myfix.tag_dict(xml_name)
        self.msg_fr = myfix.msg_types(xml_name)
        self.sessions =[]
        self.verbose = verbose
        
    def print_message(self, msg,sessionID, prepend =""):
        '''
        my function
        '''
        global debug
        global debug_exception
        feed = sessionID.toString().split("->")[1]
        try:
            msg_fr = myfix.parse_msg(msg, data_dictionary = self.data_dictionary,field_dict = self.field_dict, msg_fr = self.msg_fr)
        except Exception as e:
            print("could not parse message, saved it in debug variable and exception in debug_exception")
            debug = quickfix.Message(msg)
            debug_exception = e
            return
        msg_str = myfix.msg_frame_to_str(msg_fr)
        print(f"{feed}|{prepend}|{msg_str}")

    
    def onCreate(self, sessionID):         
        print("created", sessionID)
        self.sessions.append(sessionID)
    def onLogon(self, sessionID): 
        print("logged on", sessionID)
    def onLogout(self, sessionID):
        print("logged out", sessionID)
    def toAdmin(self, message, sessionID): 
        '''
        sending admin message from me
        '''
        if message.getHeader().getField(35) == "A": #log in message
                #message.getHeader().setField(554, "7020") # set password field
                message.setField(554, "7020") # set password field
                #message.setField(6936, "E") # set english
                

#         print("toAdmin (my msg sent):", message, "**********", sessionID)



        msg_type = message.getHeader().getField(quickfix.MsgType()).getString()
        if self.verbose:
            if msg_type == quickfix.MsgType_Heartbeat:
                print(sessionID.toString().split("->")[1] + "|heartbeat sent" )
            else:
                self.print_message(message,sessionID, "toAdmin")
    def toApp(self, message, sessionID):
        '''
        sending my actual message
        '''
#         print("toApp (sending my msg):", message, "**********", sessionID)
        if self.verbose:
            self.print_message(message,sessionID, "toApp")
    def fromAdmin(self, message, sessionID): 
        '''
        receiving admin message
        '''
#         print("fromAdmin: (receiving adming msg)", message, "**********", sessionID)
        msg_type = message.getHeader().getField(quickfix.MsgType()).getString()
        if self.verbose:
            if msg_type == quickfix.MsgType_Heartbeat:
                print(sessionID.toString().split("->")[1] + "|heartbeat received" )
            else:
                self.print_message(message,sessionID, "fromAdmin")
    def fromApp(self, message, sessionID):
        '''
        receiving actual message
        '''
#         print("fromApp: (receiving actual msg)", message, "**********", sessionID)
        if self.verbose:
            self.print_message(message,sessionID, "fromApp")

In [2]:
def send_message(msg, session  = 'IFIX-EQ-UAT'):
    candidates = [x for x in application.sessions if x.getTargetCompID().getString() == session]
    assert len(candidates) == 1, len(candidates)
    quickfix.Session.sendToTarget(msg, candidates[0])


In [3]:
application  = Application(verbose=True)

In [4]:
settings = quickfix.SessionSettings("fix_config.txt")
initiator = quickfix.SocketInitiator(application, quickfix.FileStoreFactory(settings), settings, quickfix.FileLogFactory(settings))
initiator.start()

created FIX.4.4:MU9051500001->IFIX-EQ-UAT
IFIX-EQ-UAT|toAdmin|Logon(admin): BeginString=FIX.4.4|MsgType=A|MsgSeqNum=254|SenderCompID=MU9051500001|SendingTime=20220107-19:19:18.496|TargetCompID=IFIX-EQ-UAT|EncryptMethod=0|HeartBtInt=60|Password=7020
IFIX-EQ-UAT|fromAdmin|Logon(admin): BeginString=FIX.4.4|BodyLength=86|MsgType=A|MsgSeqNum=246|SenderCompID=IFIX-EQ-UAT|SendingTime=20220107-19:19:22.836592941|TargetCompID=MU9051500001|EncryptMethod=0|HeartBtInt=60|CheckSum=088
logged on FIX.4.4:MU9051500001->IFIX-EQ-UAT


In [5]:
settings = quickfix.SessionSettings("fix_config_TC.txt")
initiator_tc = quickfix.SocketInitiator(application, quickfix.FileStoreFactory(settings), settings, quickfix.FileLogFactory(settings))
initiator_tc.start()

created FIX.4.4:MU9051500002->IFIX-TC-EQ-UAT
IFIX-TC-EQ-UAT|toAdmin|Logon(admin): BeginString=FIX.4.4|MsgType=A|MsgSeqNum=42|SenderCompID=MU9051500002|SendingTime=20220107-19:19:25.632|TargetCompID=IFIX-TC-EQ-UAT|EncryptMethod=0|HeartBtInt=60|Password=7020
IFIX-TC-EQ-UAT|fromAdmin|Logon(admin): BeginString=FIX.4.4|BodyLength=89|MsgType=A|MsgSeqNum=113|SenderCompID=IFIX-TC-EQ-UAT|SendingTime=20220107-19:19:30.772649759|TargetCompID=MU9051500002|EncryptMethod=0|HeartBtInt=60|CheckSum=033
logged on FIX.4.4:MU9051500002->IFIX-TC-EQ-UAT
IFIX-TC-EQ-UAT|fromApp|TradingSessionStatus(app): BeginString=FIX.4.4|BodyLength=142|MsgType=h|MsgSeqNum=114|SenderCompID=IFIX-TC-EQ-UAT|SendingTime=20220107-19:19:33.661663616|TargetCompID=MU9051500002|Text=Connection to MICEX market established|UnsolicitedIndicator=Y|TradingSessionID=FOND|TradSesStatus=101|CheckSum=128
IFIX-TC-EQ-UAT|fromApp|TradeCaptureReport(app): BeginString=FIX.4.4|BodyLength=518|MsgType=AE|MsgSeqNum=115|SenderCompID=IFIX-TC-EQ-UAT|Sen

IFIX-TC-EQ-UAT|fromApp|TradeCaptureReport(app): BeginString=FIX.4.4|BodyLength=518|MsgType=AE|MsgSeqNum=136|SenderCompID=IFIX-TC-EQ-UAT|SendingTime=20220107-19:19:40.647652574|TargetCompID=MU9051500002|ExecID=113731454|1|211106|LastPx=256414|LastQty=1|Symbol=AMZN-RM|TransactTime=20220107-18:11:06|SettlType=3|SettlDate=20220111|TradeDate=20220107|ExecType=F|Yield=0.0|NoSides=1|PreviouslyReported=N|TradeReportID=292|TrdType=0|LastLiquidityInd=2|CalculatedCcyLastQty=1|InstitutionID=NCC|OptionSettlType=Y2|CurrencyCode=SUR|MOEXStipulationValue=N|ClientAccID=11866|Side=1|OrderID=28092528710|ClOrdID=2076087002/|NoPartyIDs=3|Account=L00-000012D8|TradingSessionID=FQBR|TradingSessionSubID=N|Commission=25.64|CommType=3|GrossTradeAmt=256414.0|AccruedInterestAmt=0.0|NoMiscFees=3|MiscFeeAmt=10.9|MiscFeeType=98|MiscFeeAmt=14.74|MiscFeeType=4|MiscFeeAmt=0.0|MiscFeeType=99|PartyID=MC9051500000|PartyIDSource=D|PartyRole=1|PartyID=2076087002|PartyIDSource=D|PartyRole=3|PartyID=MU9051500001|PartyIDSource=

In [8]:
settings = quickfix.SessionSettings("fix_config_DC.txt")
initiator_dc = quickfix.SocketInitiator(application, quickfix.FileStoreFactory(settings), settings, quickfix.FileLogFactory(settings))
initiator_dc.start()

created FIX.4.4:MU9051500004->IFIX-DC-EQ-UAT
IFIX-DC-EQ-UAT|toAdmin|Logon(admin): BeginString=FIX.4.4|MsgType=A|MsgSeqNum=1|SenderCompID=MU9051500004|SendingTime=20220107-19:20:51.582|TargetCompID=IFIX-DC-EQ-UAT|EncryptMethod=0|HeartBtInt=60|Password=7020
IFIX-DC-EQ-UAT|fromAdmin|Logon(admin): BeginString=FIX.4.4|BodyLength=87|MsgType=A|MsgSeqNum=1|SenderCompID=IFIX-DC-EQ-UAT|SendingTime=20220107-19:20:56.732333551|TargetCompID=MU9051500004|EncryptMethod=0|HeartBtInt=60|CheckSum=149
logged on FIX.4.4:MU9051500004->IFIX-DC-EQ-UAT
IFIX-DC-EQ-UAT|fromApp|ExecutionReport(app): BeginString=FIX.4.4|BodyLength=378|MsgType=8|MsgSeqNum=2|SenderCompID=IFIX-DC-EQ-UAT|SendingTime=20220107-19:21:06.731339464|TargetCompID=MU9051500004|Account=L00-000012D8|AvgPx=0|ClOrdID=2076087002/|CumQty=0|ExecID=2076087002/|0|28092442643|LastPx=0|LastQty=0|OrderID=28092442643|OrderQty=1|OrdStatus=0|OrdType=1|Side=1|Symbol=AMZN-RM|TransactTime=20220107-07:37:00|ExecType=0|LeavesQty=1|MDEntryID=23275|TradingSession

IFIX-DC-EQ-UAT|fromApp|ExecutionReport(app): BeginString=FIX.4.4|BodyLength=540|MsgType=8|MsgSeqNum=25|SenderCompID=IFIX-DC-EQ-UAT|SendingTime=20220107-19:21:06.734210343|TargetCompID=MU9051500004|Account=L00-000012D8|AvgPx=0|ClOrdID=lorcculqqlghzdfyxkrg|Commission=25.69|CommType=3|CumQty=1|ExecID=113723856|B|112311|LastPx=256953|LastQty=1|OrderID=28092453999|OrderQty=1|OrdStatus=2|OrdType=1|Side=1|Symbol=AMZN-RM|TransactTime=20220107-08:23:11|SettlDate=20220111|NoMiscFees=3|ExecType=F|LeavesQty=0|AccruedInterestAmt=0.0|Yield=0.0|MDEntryID=29209|TradingSessionID=FQBR|NoPartyIDs=3|TradingSessionSubID=NORM|LastLiquidityInd=2|OptionSettlDate=20220111|InstitutionID=NCC|OptionSettlType=Y2|CurrencyCode=SUR|MOEXStipulationValue=N|ClientAccID=11866|MOEXOrigTime=337676|PreMatchedCumQty=0|MiscFeeAmt=10.92|MiscFeeType=98|MiscFeeAmt=14.77|MiscFeeType=4|MiscFeeAmt=0.0|MiscFeeType=99|PartyID=MC9051500000|PartyIDSource=D|PartyRole=1|PartyID=2076087002|PartyIDSource=D|PartyRole=3|PartyID=MU9051500001|

IFIX-EQ-UAT|heartbeat sent
IFIX-EQ-UAT|heartbeat received


In [17]:
def stop():
    initiator.stop()
    initiator_tc.stop()
    initiator_dc.stop()

In [10]:
order = fixtrading.create_order("AMZN-RM", "buy", None, 1, board = "FQBR")
send_message(order)

IFIX-EQ-UAT|toApp|NewOrderSingle(app): BeginString=FIX.4.4|MsgType=D|MsgSeqNum=258|SenderCompID=MU9051500001|SendingTime=20220107-19:22:27.655|TargetCompID=IFIX-EQ-UAT|Account=L00-000012D8|ClOrdID=bjdnihwigyffpyvpdiag|OrderQty=1|OrdType=1|Side=1|Symbol=AMZN-RM|NoTradingSessions=1|NoPartyIDs=1|TradingSessionID=FQBR|PartyRole=3|PartyIDSource=D|PartyID=2076087002
IFIX-EQ-UAT|fromApp|ExecutionReport(app): BeginString=FIX.4.4|BodyLength=389|MsgType=8|MsgSeqNum=250|SenderCompID=IFIX-EQ-UAT|SendingTime=20220107-19:22:29.903418057|TargetCompID=MU9051500001|Account=L00-000012D8|AvgPx=0|ClOrdID=bjdnihwigyffpyvpdiag|CumQty=0|ExecID=bjdnihwigyffpyvpdiag|0|28092528713|OrderID=28092528713|OrderQty=1|OrdStatus=0|OrdType=1|Side=1|Symbol=AMZN-RM|Text=(162) Buy order #28092528713 accepted (1 matched)|TransactTime=20220107-19:22:29|ExecType=0|LeavesQty=1|MDEntryID=68258|TradingSessionID=FQBR|NoPartyIDs=1|RequestTime=20220107-19:22:29.901010326|MOEXOrigTime=901718|PartyID=2076087002|PartyIDSource=D|PartyR

In [12]:
df = myfix.get_messages(myfix.TC_FEED)

100%|██████████| 193/193 [00:01<00:00, 153.49it/s]


In [32]:
df = df.query("msgtype=='AE'")
df = df[df['msg_timestamp']>='2022-01-07 19:19:38.696096']
myfix.pivot_msgtype(df)[['Symbol', 'TransactTime', 'Side0', 'LastPx','LastQty']]#['ExecID'].value_counts()

,Symbol,TransactTime,Side0,LastPx,LastQty
msg_num,,,,,
157,AMZN-RM,20220107-07:37:00,1,256414,1
158,AMZN-RM,20220107-07:53:15,1,256414,1
159,AMZN-RM,20220107-07:57:41,1,257493,1
160,AMZN-RM,20220107-08:08:07,1,256414,1
161,AMZN-RM,20220107-08:10:05,1,256953,1
162,AMZN-RM,20220107-08:12:34,1,257493,1
163,AMZN-RM,20220107-08:14:29,1,256953,1
164,AMZN-RM,20220107-08:15:35,1,256414,1
165,AMZN-RM,20220107-08:16:38,1,257493,1


In [11]:
initiator_tc.stop()

IFIX-TC-EQ-UAT|toAdmin|Logout(admin): BeginString=FIX.4.4|MsgType=5|MsgSeqNum=316|SenderCompID=MU9051500001|SendingTime=20220107-18:30:13.025|TargetCompID=IFIX-TC-EQ-UAT
IFIX-TC-EQ-UAT|fromAdmin|Logout(admin): BeginString=FIX.4.4|BodyLength=103|MsgType=5|MsgSeqNum=424|SenderCompID=IFIX-TC-EQ-UAT|SendingTime=20220107-18:30:15.273627081|TargetCompID=MU9051500001|Text=The confirming Logout.|CheckSum=231
logged out FIX.4.4:MU9051500001->IFIX-TC-EQ-UAT


In [11]:
import xml.etree.ElementTree as ET


IFIX-TC-EQ-UAT|heartbeat sent


In [56]:
importlib.reload(myfix)

<module 'myfix' from '/home/gpashchenko/projects/fix/myfix.py'>

In [30]:
df2 = myfix.get_messages(myfix.TC_FEED)

100%|██████████| 407/407 [00:02<00:00, 167.78it/s]


In [57]:
fr = myfix.pivot_msgtype(df2.query("msgtype=='AE'"))

In [7]:
df = myfix.get_messages(myfix.DC_FEED)

48it [00:00, 152.42it/s]


In [20]:
df = df.query('msgtype=="8"')
df['unique_key'] = np.where(df['group_id'].isna(), df['key_str'], df['key_str'] +df['group_id'].astype(str))
df.pivot('msg_num', 'unique_key', 'val')

unique_key,Account,AccruedInterestAmt,AvgPx,BeginString,BodyLength,CheckSum,ClOrdID,ClientAccID,CommType,Commission,CumQty,CurrencyCode,ExecID,ExecType,InstitutionID,LastLiquidityInd,LastPx,LastQty,LeavesQty,MDEntryID,MOEXOrigTime,MOEXStipulationValue,MiscFeeAmt0.0,MiscFeeAmt1.0,MiscFeeAmt2.0,MiscFeeType0.0,MiscFeeType1.0,MiscFeeType2.0,MsgSeqNum,MsgType,NoMiscFees,NoPartyIDs,OptionSettlDate,OptionSettlType,OrdStatus,OrdType,OrderID,OrderQty,PartyID0.0,PartyID1.0,PartyID2.0,PartyID3.0,PartyID4.0,PartyID5.0,PartyIDSource0.0,PartyIDSource1.0,PartyIDSource2.0,PartyIDSource3.0,PartyIDSource4.0,PartyIDSource5.0,PartyRole0.0,PartyRole1.0,PartyRole2.0,PartyRole3.0,PartyRole4.0,PartyRole5.0,PreMatchedCumQty,Price,PriceType,SenderCompID,SendingTime,SettlDate,Side,Symbol,TargetCompID,TradingSessionID,TradingSessionSubID,TransactTime,Yield
msg_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,L00-000012D8,NaN,0,FIX.4.4,378,253,2076087002/,NaN,NaN,NaN,0,NaN,2076087002/|0|28092442643,0,NaN,NaN,0,0,1,23275,767892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,8,NaN,3,NaN,NaN,0,1,28092442643,1,MC9051500000,2076087002,MU9051500003,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,NaN,NaN,NaN,0,NaN,2,IFIX-DC-EQ-UAT,20220107-09:12:59.343487195,NaN,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:37:00,NaN
8,L00-000012D8,0.0,0,FIX.4.4,529,100,2076087002/,11866,3,25.64,1,SUR,113722727|B|103700,F,NCC,2,256414,1,0,23275,767892,N,10.9,14.74,0.0,98,4,99,4,8,3,3,20220111,Y2,2,1,28092442643,1,NaN,NaN,NaN,MC9051500000,2076087002,MU9051500003,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,0,NaN,NaN,IFIX-DC-EQ-UAT,20220107-09:12:59.344578392,20220111,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:37:00,0.0
9,L00-000012D8,NaN,0,FIX.4.4,396,128,plomcxldlkqbvekgxlco,NaN,NaN,NaN,0,NaN,plomcxldlkqbvekgxlco|0|28092446529,0,NaN,NaN,0,0,1,25243,586151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,8,NaN,3,NaN,NaN,0,1,28092446529,1,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,NaN,NaN,NaN,0,NaN,2,IFIX-DC-EQ-UAT,20220107-09:12:59.344807971,NaN,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:53:15,NaN
10,L00-000012D8,0.0,0,FIX.4.4,538,161,plomcxldlkqbvekgxlco,11866,3,25.64,1,SUR,113723093|B|105315,F,NCC,2,256414,1,0,25243,586151,N,10.9,14.74,0.0,98,4,99,6,8,3,3,20220111,Y2,2,1,28092446529,1,NaN,NaN,NaN,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,0,NaN,NaN,IFIX-DC-EQ-UAT,20220107-09:12:59.344875725,20220111,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:53:15,0.0
11,L00-000012D8,NaN,0,FIX.4.4,396,155,mdrakbomrkzvsfinijlp,NaN,NaN,NaN,0,NaN,mdrakbomrkzvsfinijlp|0|28092447639,0,NaN,NaN,0,0,1,25820,008920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,8,NaN,3,NaN,NaN,0,1,28092447639,1,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,NaN,NaN,NaN,0,NaN,2,IFIX-DC-EQ-UAT,20220107-09:12:59.344957843,NaN,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:57:41,NaN
12,L00-000012D8,0.0,0,FIX.4.4,539,244,mdrakbomrkzvsfinijlp,11866,3,25.75,1,SUR,113723196|B|105741,F,NCC,2,257493,1,0,25820,008920,N,10.94,14.81,0.0,98,4,99,8,8,3,3,20220111,Y2,2,1,28092447639,1,NaN,NaN,NaN,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,0,NaN,NaN,IFIX-DC-EQ-UAT,20220107-09:12:59.345077986,20220111,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-07:57:41,0.0
13,L00-000012D8,NaN,0,FIX.4.4,396,119,mtfrgystwkrkmhabdtfe,NaN,NaN,NaN,0,NaN,mtfrgystwkrkmhabdtfe|0|28092450261,0,NaN,NaN,0,0,1,27204,179045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,8,NaN,3,NaN,NaN,0,1,28092450261,1,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,NaN,NaN,NaN,0,NaN,2,IFIX-DC-EQ-UAT,20220107-09:12:59.345155307,NaN,1,AMZN-RM,MU9051500001,FQBR,NORM,20220107-08:08:07,NaN
14,L00-000012D8,0.0,0,FIX.4.4,539,192,mtfrgystwkrkmhabdtfe,11866,3,25.64,1,SUR,113723447|B|110807,F,NCC,2,256414,1,0,27204,179045,N,10.9,14.74,0.0,98,4,99,10,8,3,3,20220111,Y2,2,1,28092450261,1,NaN,NaN,NaN,MC9051500000,2076087002,MU9051500001,NaN,NaN,NaN,D,D,D,NaN,NaN,NaN,1,3,12,0,NaN,NaN,IFIX-DC-EQ-UAT,20220107-09:12

IFIX-DC-EQ-UAT|heartbeat sent
IFIX-DC-EQ-UAT|heartbeat received
IFIX-DC-EQ-UAT|heartbeat sent
IFIX-DC-EQ-UAT|heartbeat received
IFIX-DC-EQ-UAT|heartbeat sent
IFIX-DC-EQ-UAT|heartbeat received


IFIX-EQ-UAT|fromAdmin|Logon(admin): BeginString=FIX.4.4|BodyLength=85|MsgType=A|MsgSeqNum=86|SenderCompID=IFIX-EQ-UAT|SendingTime=20220107-08:30:26.486093627|TargetCompID=MU9051500001|EncryptMethod=0|HeartBtInt=60|CheckSum=034
logged on FIX.4.4:MU9051500001->IFIX-EQ-UAT


In [29]:
msg = fixtrading.create_order("AMZN-RM", "sell", 224044, 1, board = "FQBR" )

In [30]:
send_message(msg)

IFIX-EQ-UAT|toApp|NewOrderSingle(app): BeginString=FIX.4.4|MsgType=D|MsgSeqNum=148|SenderCompID=MU9051500001|SendingTime=20220107-09:07:42.647|TargetCompID=IFIX-EQ-UAT|Account=L00-000012D8|ClOrdID=hqfrtvzrwfokfazfqiwy|OrderQty=1|OrdType=2|Price=224044|Side=2|Symbol=AMZN-RM|NoTradingSessions=1|NoPartyIDs=1|TradingSessionID=FQBR|PartyRole=3|PartyIDSource=D|PartyID=2076087002
IFIX-EQ-UAT|fromApp|ExecutionReport(app): BeginString=FIX.4.4|BodyLength=400|MsgType=8|MsgSeqNum=134|SenderCompID=IFIX-EQ-UAT|SendingTime=20220107-09:07:44.791089122|TargetCompID=MU9051500001|Account=L00-000012D8|AvgPx=0|ClOrdID=hqfrtvzrwfokfazfqiwy|CumQty=0|ExecID=hqfrtvzrwfokfazfqiwy|0|28092466847|OrderID=28092466847|OrderQty=1|OrdStatus=0|OrdType=2|Price=224044|Side=2|Symbol=AMZN-RM|Text=(163) Sell order #28092466847 accepted (1 matched)|TransactTime=20220107-09:07:44|ExecType=0|LeavesQty=1|MDEntryID=36242|TradingSessionID=FQBR|NoPartyIDs=1|RequestTime=20220107-09:07:44.789184943|MOEXOrigTime=789892|PartyID=207608

In [13]:
print(debug.toXML())

<message>
  <header>
    <field number="8"><![CDATA[FIX.4.4]]></field>
    <field number="9"><![CDATA[439]]></field>
    <field number="35"><![CDATA[8]]></field>
    <field number="34"><![CDATA[78]]></field>
    <field number="49"><![CDATA[IFIX-EQ-UAT]]></field>
    <field number="52"><![CDATA[20220107-08:23:11.342073850]]></field>
    <field number="56"><![CDATA[MU9051500001]]></field>
  </header>
  <body>
    <field number="1"><![CDATA[L00-000012D8]]></field>
    <field number="6"><![CDATA[0]]></field>
    <field number="11"><![CDATA[lorcculqqlghzdfyxkrg]]></field>
    <field number="14"><![CDATA[1]]></field>
    <field number="17"><![CDATA[113723856|B|112311]]></field>
    <field number="31"><![CDATA[256953]]></field>
    <field number="32"><![CDATA[1]]></field>
    <field number="37"><![CDATA[28092453999]]></field>
    <field number="38"><![CDATA[1]]></field>
    <field number="39"><![CDATA[2]]></field>
    <field number="40"><![CDATA[1]]></field>
    <field number="54"><![CDATA[1]

In [8]:
df = myfix.get_messages(myfix.TRADE_FEED)

36it [00:00, 145.84it/s]


In [17]:
myfix.parse_msg("8=FIX.4.49=8735=A34=149=MU905150000152=20220106-06:36:43.70856=IFIX-EQ-UAT554=702098=0108=6010=080")

,key,key_str,val,msg_part,msgtype,msg_time,name,msgcat
0,8,BeginString,FIX.4.4,header,A,2022-01-06 06:36:43.708,Logon,admin
1,9,BodyLength,87,header,A,2022-01-06 06:36:43.708,Logon,admin
2,35,MsgType,A,header,A,2022-01-06 06:36:43.708,Logon,admin
3,34,MsgSeqNum,1,header,A,2022-01-06 06:36:43.708,Logon,admin
4,49,SenderCompID,MU9051500001,header,A,2022-01-06 06:36:43.708,Logon,admin
5,52,SendingTime,20220106-06:36:43.708,header,A,2022-01-06 06:36:43.708,Logon,admin
6,56,TargetCompID,IFIX-EQ-UAT,header,A,2022-01-06 06:36:43.708,Logon,admin
7,98,EncryptMethod,0,body,A,2022-01-06 06:36:43.708,Logon,admin
8,108,HeartBtInt,60,body,A,2022-01-06 06:36:43.708,Logon,admin
9,554,Password,7020,body,A,2022-01-06 06:36:43.708,Logon,admin


IFIX-EQ-UAT|toAdmin|TestRequest(admin): BeginString=FIX.4.4|MsgType=1|MsgSeqNum=5|SenderCompID=MU9051500001|SendingTime=20220106-06:39:09.497|TargetCompID=IFIX-EQ-UAT|TestReqID=TEST
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=6|SenderCompID=MU9051500001|SendingTime=20220106-06:40:09.305|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=7|SenderCompID=MU9051500001|SendingTime=20220106-06:41:09.044|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=8|SenderCompID=MU9051500001|SendingTime=20220106-06:42:09.385|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=9|SenderCompID=MU9051500001|SendingTime=20220106-06:43:09.308|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginStrin

In [17]:
myfix.parse_msg("8=FIX.4.49=11835=334=17049=MU905150000152=20220106-08:52:31.18456=IFIX-EQ-UAT45=14958=Invalid tag number371=5979372=8373=010=158")

,key,key_str,val,msg_part,msgtype,msg_time,sender,name,msgcat
0,8,BeginString,FIX.4.4,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
1,9,BodyLength,118,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
2,35,MsgType,3,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
3,34,MsgSeqNum,170,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
4,49,SenderCompID,MU9051500001,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
5,52,SendingTime,20220106-08:52:31.184,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
6,56,TargetCompID,IFIX-EQ-UAT,header,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
7,45,RefSeqNum,149,body,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
8,58,Text,Invalid tag number,body,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin
9,371,RefTagID,5979,body,3,2022-01-06 08:52:31.184,MU9051500001,Reject,admin


IFIX-EQ-UAT|toAdmin|TestRequest(admin): BeginString=FIX.4.4|MsgType=1|MsgSeqNum=171|SenderCompID=MU9051500001|SendingTime=20220106-08:53:22.003|TargetCompID=IFIX-EQ-UAT|TestReqID=TEST
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=172|SenderCompID=MU9051500001|SendingTime=20220106-08:54:22.123|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=173|SenderCompID=MU9051500001|SendingTime=20220106-08:55:22.021|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=174|SenderCompID=MU9051500001|SendingTime=20220106-08:56:22.092|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=175|SenderCompID=MU9051500001|SendingTime=20220106-08:57:22.120|TargetCompID=IFIX-EQ-UAT


In [86]:
'''
send order
'''
import quickfix


order_count = 0



message = quickfix.Message()
header = message.getHeader()
trailer = message.getTrailer() 

#header.setField(quickfix.BeginString(quickfix.BeginString_FIX44)) #

header.setField(quickfix.MsgType(quickfix.MsgType_NewOrderSingle))
message.setField(quickfix.ClOrdID(str(order_count)))
message.setField(quickfix.Account("L00-000012D8")) #торговый счет




group = quickfix.Group(386, 336) #no trading sessions, and tradingsessionId
group.setField(quickfix.NoTradingSessions(1))

group.setField(quickfix.TradingSessionID("TQBR"))
message.addGroup(group)

# message.setField(quickfix.NoTradingSessions(1))
# message.setField(quickfix.TradingSessionID("TQBR")) #secboard








message.setField(quickfix.Symbol("SBER")) #seccode
message.setField(quickfix.Side(quickfix.Side_BUY)) #side


message.setField(quickfix.OrderQty(1)) #В ЛОТАХ!!

message.setField(quickfix.OrdType(quickfix.OrdType_MARKET)) #


transact_time = quickfix.TransactTime()
ts = datetime.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S.%f")[:-3]
transact_time.setString(ts)

message.setField(transact_time)


message.setField(quickfix.SecondaryClOrdID("2076087002"))



# message.setField(quickfix.ClOrdID("321"))
# message.setField(quickfix.Symbol("LNUX"))
# message.setField(quickfix.Side("1"))
# message.setField(quickfix.Text("Cancel My Order!"))


print(message.toXML())




<message>
  <header>
    <field number="35"><![CDATA[D]]></field>
  </header>
  <body>
    <field number="1"><![CDATA[L00-000012D8]]></field>
    <field number="11"><![CDATA[0]]></field>
    <field number="38"><![CDATA[1]]></field>
    <field number="40"><![CDATA[1]]></field>
    <field number="54"><![CDATA[1]]></field>
    <field number="55"><![CDATA[SBER]]></field>
    <field number="60"><![CDATA[20220106-08:28:36.660]]></field>
    <field number="386"><![CDATA[1]]></field>
    <field number="526"><![CDATA[2076087002]]></field>
    <group>
      <field number="336"><![CDATA[TQBR]]></field>
      <field number="386"><![CDATA[1]]></field>
    </group>
  </body>
  <trailer>
  </trailer>
</message>


IFIX-EQ-UAT|toApp|NewOrderSingle(app): BeginString=FIX.4.4|MsgType=D|MsgSeqNum=23|SenderCompID=MU9051500001|SendingTime=20220106-07:00:30.872|TargetCompID=IFIX-EQ-UAT|Account=L00-000012D8|ClOrdID=0|OrderQty=1|OrdType=1|Side=1|Symbol=SBER|TransactTime=20220106-06:59:52.077|TradingSessionID=TQBR|NoTradingSessions=1|CheckSum=246


True

IFIX-EQ-UAT|fromAdmin|Reject(admin): BeginString=FIX.4.4|BodyLength=275|MsgType=3|MsgSeqNum=21|SenderCompID=IFIX-EQ-UAT|SendingTime=20220106-07:00:32.212062482|TargetCompID=MU9051500001|RefSeqNum=23|Text=Last tag in the message root is not CheckSum. Tag out of order. Expected: 10. In fact: 386. Parsing stopped at column: 173 in tag 386 in message D with sequence number 23.|RefTagID=386|RefMsgType=D|SessionRejectReason=14|CheckSum=110
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=24|SenderCompID=MU9051500001|SendingTime=20220106-07:01:30.216|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=25|SenderCompID=MU9051500001|SendingTime=20220106-07:02:30.216|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|heartbeat
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=26|SenderCompID=MU9051500001|SendingTime=20220106-07:03:30.218|TargetCompID=IFIX-EQ-UA

In [21]:
myfix.msg_types().query("msgtype=='D'")

,name,msgtype,msgcat
13,NewOrderSingle,D,app


IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=154|SenderCompID=MU9051500002|SendingTime=20220105-18:08:13.330|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=154|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:08:14.153028334|TargetCompID=MU9051500002|CheckSum=017


In [49]:
with open(myfix.TRADE_FEED) as f:
    line = f.read()

In [53]:
msg = quickfix.Message("8=FIX.4.4\x019=78\x0135=A\x0134=1\x0149=MU9051500001\x0152=20220105-09:48:09.417\x0156=IFIX-EQ-UAT\x0198=0\x01108=20\x0110=171\x018=FIX.4.4\x019=78\x0135=A\x0134=2\x0149=MU9051500001\x0152=20220105-09:49:09.602\x0156=IFIX-EQ-UAT\x0198=0\x01108=20\x0110=169\x01", application.data_dictionary)
print(msg)

InvalidMessage: Invalid message: Expected BodyLength=156, Received BodyLength=78

IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=219|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:29:54.219749102|TargetCompID=MU9051500002|CheckSum=032
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=219|SenderCompID=MU9051500002|SendingTime=20220105-18:29:53.053|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=220|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:30:14.220617275|TargetCompID=MU9051500002|CheckSum=009
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=220|SenderCompID=MU9051500002|SendingTime=20220105-18:30:13.083|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=221|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:30:34.222519230|TargetCompID=MU9051500002|CheckSum=006
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|Ms

IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=242|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:37:34.241545389|TargetCompID=MU9051500002|CheckSum=031
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=242|SenderCompID=MU9051500002|SendingTime=20220105-18:37:33.084|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=243|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:37:54.243489309|TargetCompID=MU9051500002|CheckSum=035
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=243|SenderCompID=MU9051500002|SendingTime=20220105-18:37:53.085|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=244|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:38:14.246615061|TargetCompID=MU9051500002|CheckSum=022
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|Ms

In [45]:
field_names = myfix.data_dict()
field_names

{'1': 'Account',
 '2': 'AdvId',
 '3': 'AdvRefID',
 '4': 'AdvSide',
 '5': 'AdvTransType',
 '6': 'AvgPx',
 '7': 'BeginSeqNo',
 '8': 'BeginString',
 '9': 'BodyLength',
 '10': 'CheckSum',
 '11': 'ClOrdID',
 '12': 'Commission',
 '13': 'CommType',
 '14': 'CumQty',
 '15': 'Currency',
 '16': 'EndSeqNo',
 '17': 'ExecID',
 '18': 'ExecInst',
 '19': 'ExecRefID',
 '21': 'HandlInst',
 '22': 'SecurityIDSource',
 '23': 'IOIID',
 '25': 'IOIQltyInd',
 '26': 'IOIRefID',
 '27': 'IOIQty',
 '28': 'IOITransType',
 '29': 'LastCapacity',
 '30': 'LastMkt',
 '31': 'LastPx',
 '32': 'LastQty',
 '33': 'NoLinesOfText',
 '34': 'MsgSeqNum',
 '35': 'MsgType',
 '36': 'NewSeqNo',
 '37': 'OrderID',
 '38': 'OrderQty',
 '39': 'OrdStatus',
 '40': 'OrdType',
 '41': 'OrigClOrdID',
 '42': 'OrigTime',
 '43': 'PossDupFlag',
 '44': 'Price',
 '45': 'RefSeqNum',
 '48': 'SecurityID',
 '49': 'SenderCompID',
 '50': 'SenderSubID',
 '52': 'SendingTime',
 '53': 'Quantity',
 '54': 'Side',
 '55': 'Symbol',
 '56': 'TargetCompID',
 '57': 'Tar

IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=208|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:26:14.210268781|TargetCompID=MU9051500002|CheckSum=023
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=208|SenderCompID=MU9051500002|SendingTime=20220105-18:26:13.088|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=209|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:26:34.210358631|TargetCompID=MU9051500002|CheckSum=020
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=209|SenderCompID=MU9051500002|SendingTime=20220105-18:26:33.088|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=210|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:26:54.211636136|TargetCompID=MU9051500002|CheckSum=014
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|Ms

    def put_order(self, sessionID, myinstrument, myquantity):
        self.myinstrument = myinstrument
        self.myquantity = myquantity
        print("Creating the following order: ")
        today = datetime.datetime.now()
        nextID = today.strftime("%m%d%Y%H%M%S%f")
        trade = fix.Message()
        trade.getHeader().setField(fix.StringField(8, "FIX.4.4"))
        trade.getHeader().setField(fix.MsgType(fix.MsgType_NewOrderSingle))
        trade.setField(fix.ClOrdID(nextID)) #11=Unique order
        trade.getHeader().setField(fix.Account("account"))
        trade.getHeader().setField(fix.TargetSubID("targetsubid"))
        trade.setField(fix.Symbol(myinstrument)) #55=SMBL ?
        trade.setField(fix.TransactTime())
        trade.setField(fix.CharField(54, fix.Side_BUY))
        trade.setField(fix.OrdType(fix.OrdType_MARKET))  # 40=2 Limit order
        trade.setField(fix.OrderQty(myquantity))  # 38=100
        print trade.toString()
        fix.Session.sendToTarget(trade, self.sessionID)

In [34]:
[x for x in dir(quickfix) if "44" in x.lower()]

['ApplVerID_FIX44', 'BeginString_FIX44']

IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=183|SenderCompID=MU9051500002|SendingTime=20220105-18:17:53.000|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=183|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:17:54.178773117|TargetCompID=MU9051500002|CheckSum=036
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=184|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:18:14.181885594|TargetCompID=MU9051500002|CheckSum=041
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=184|SenderCompID=MU9051500002|SendingTime=20220105-18:18:13.061|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=185|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-18:18:34.185060409|TargetCompID=MU9051500002|CheckSum=028
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|Ms

In [14]:
cur_session = initiator.getSession(application.sessions[0])

cur_session

<quickfix.Session; proxy of <Swig Object of type 'FIX::Session *' at 0x7f62076c1270> >

IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=126|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:58:54.115457251|TargetCompID=MU9051500002|CheckSum=026
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=126|SenderCompID=MU9051500002|SendingTime=20220105-17:58:53.033|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=127|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:59:14.117621298|TargetCompID=MU9051500002|CheckSum=030
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=127|SenderCompID=MU9051500002|SendingTime=20220105-17:59:13.018|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=128|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:59:34.118750565|TargetCompID=MU9051500002|CheckSum=034
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|Ms

In [34]:
session = initiator.getSession(quickfix.SessionID("8=FIX.4.4", "MU9051500001", "IFIX-EQ-UAT"))
session

IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=113|SenderCompID=MU9051500002|SendingTime=20220105-17:54:35.105|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=111|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:54:36.610957228|TargetCompID=MU9051500002|CheckSum=025


In [38]:
s = initiator.getSessions()

In [32]:
initiator.m_sessions

AttributeError: 'SocketInitiatorBase' object has no attribute 'm_sessions'

IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=111|SenderCompID=MU9051500002|SendingTime=20220105-17:53:55.306|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=109|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:53:56.608699860|TargetCompID=MU9051500002|CheckSum=045
IFIX-EQ-UAT|toAdmin|Heartbeat(admin): BeginString=FIX.4.4|MsgType=0|MsgSeqNum=112|SenderCompID=MU9051500002|SendingTime=20220105-17:54:15.153|TargetCompID=IFIX-EQ-UAT
IFIX-EQ-UAT|fromAdmin|Heartbeat(admin): BeginString=FIX.4.4|BodyLength=74|MsgType=0|MsgSeqNum=110|SenderCompID=IFIX-EQ-UAT|SendingTime=20220105-17:54:16.609820265|TargetCompID=MU9051500002|CheckSum=020


In [30]:
session

In [10]:
initiator.stop()

toAdmin (my msg sent): 8=FIX.4.49=6735=534=4349=MU905150000252=20220105-17:30:35.41256=IFIX-EQ-UAT10=172 ********** FIX.4.4:MU9051500002->IFIX-EQ-UAT
fromAdmin: (receiving adming msg) 8=FIX.4.49=9935=534=4149=IFIX-EQ-UAT52=20220105-17:30:36.85366204656=MU905150000258=The confirming Logout.10=209 ********** FIX.4.4:MU9051500002->IFIX-EQ-UAT
logged out FIX.4.4:MU9051500002->IFIX-EQ-UAT


In [6]:
print(debug)

9=010=167


In [7]:
debug.toString()

'9=0\x0110=167\x01'

In [10]:
type(debug.toString())

str

toAdmin (my msg sent): 8=FIX.4.49=6735=034=3049=MU905150000252=20220105-17:22:47.02656=IFIX-EQ-UAT10=168 ********** FIX.4.4:MU9051500002->IFIX-EQ-UAT
hi


In [7]:
msg = "8=FIX.4.49=8135=234=249=IFIX-EQ-UAT52=20220105-10:06:44.29728505256=MU90515000017=116=010=039"
p = myfix.parse_msg(msg)

toAdmin (my msg sent): 8=FIX.4.49=6735=034=2449=MU905150000152=20220105-10:08:03.38656=IFIX-EQ-UAT10=168 ********** FIX.4.4:MU9051500001->IFIX-EQ-UAT


# parse message

In [252]:
importlib.reload(myfix)

<module 'myfix' from '/home/gpashchenko/projects/fix/myfix.py'>

In [254]:


with open(myfix.TRADE_FEED) as f:
    line = f.read()

In [255]:
df = myfix.get_messages(line)

12it [00:00, 168.92it/s]


In [256]:
df

,key,key_str,val,msg_part,msgtype,msg_time,name,msgcat,msg_num
0,8,BeginString,FIX.4.4,header,A,2021-12-26 00:01:00.002,Logon,admin,0
1,9,BodyLength,78,header,A,2021-12-26 00:01:00.002,Logon,admin,0
2,35,MsgType,A,header,A,2021-12-26 00:01:00.002,Logon,admin,0
3,34,MsgSeqNum,1,header,A,2021-12-26 00:01:00.002,Logon,admin,0
4,49,SenderCompID,MU9051500001,header,A,2021-12-26 00:01:00.002,Logon,admin,0
...,...,...,...,...,...,...,...,...,...
115,52,SendingTime,20211226-00:11:59.622,header,A,2021-12-26 00:11:59.622,Logon,admin,11
116,56,TargetCompID,IFIX-EQ-UAT,header,A,2021-12-26 00:11:59.622,Logon,admin,11
117,98,EncryptMethod,0,body,A,2021-12-26 00:11:59.622,Logon,admin,11
118,108,HeartBtInt,20,body,A,2021-12-26 00:11:59.622,Logon,admin,11


In [240]:
df.query('msg_num==1')

,key,key_str,val,msg_part,msgtype,msg_time,name,msgcat,msg_num
10,8,BeginString,FIX.4.4,header,A,20211226-00:01:59.489,Logon,admin,1
11,9,BodyLength,81,header,A,20211226-00:01:59.489,Logon,admin,1
12,35,MsgType,A,header,A,20211226-00:01:59.489,Logon,admin,1
13,34,MsgSeqNum,2,header,A,20211226-00:01:59.489,Logon,admin,1
14,49,SenderCompID,MU9051500001,header,A,20211226-00:01:59.489,Logon,admin,1
15,52,SendingTime,20211226-00:01:59.489,header,A,20211226-00:01:59.489,Logon,admin,1
16,56,TargetCompID,IFIX-DC-EQ-UAT,header,A,20211226-00:01:59.489,Logon,admin,1
17,98,EncryptMethod,0,body,A,20211226-00:01:59.489,Logon,admin,1
18,108,HeartBtInt,20,body,A,20211226-00:01:59.489,Logon,admin,1
19,10,CheckSum,088,trailer,A,20211226-00:01:59.489,Logon,admin,1
